In [28]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler
                                  )

In [3]:
# Step 1: Data Set
df = pd.DataFrame({'col_a': [1,2,np.nan,1,2,1.5,1,np.nan],
                   'col_b': ['cat','dog',np.nan,'dog','dog',np.nan,'dog','cat'],
                   'col_c':[1,3,3,3,7,3,1,3]})
df = df.astype({'col_b':'category'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   col_a   6 non-null      float64 
 1   col_b   6 non-null      category
 2   col_c   8 non-null      int64   
dtypes: category(1), float64(1), int64(1)
memory usage: 392.0 bytes


In [4]:
df

,col_a,col_b,col_c
0,1.0,cat,1
1,2.0,dog,3
2,NaN,NaN,3
3,1.0,dog,3
4,2.0,dog,7
5,1.5,NaN,3
6,1.0,dog,1
7,NaN,cat,3


In [7]:
# Split DS for train and test
df_train = df.iloc[:5, :]
df_test = df.iloc[5:, :]

display(df_train)
display(df_test)

,col_a,col_b,col_c
0,1.0,cat,1
1,2.0,dog,3
2,NaN,NaN,3
3,1.0,dog,3
4,2.0,dog,7


,col_a,col_b,col_c
5,1.5,NaN,3
6,1.0,dog,1
7,NaN,cat,3


# Imputer
## For Df_train
- Firts column 'col_a' have 1 NaN => take mean and imputer value = 1.2
- In 'column_b' have a 1 NaN => 1 possibility is use the frequenly valuue = dog

## For Df_test
- In 'columna' Have a 1 NaN
- In 'column_b' have a 1 NaN =>

In [12]:
# Get columns
num_cols = df_train.select_dtypes('number').columns
cat_cols = df_train.select_dtypes('category').columns

In [35]:
cat_proc = Pipeline([
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first')) # ignore unknow
])
nnum_proc = Pipeline([
    ('nimputer', SimpleImputer(strategy= 'mean')),
    ('scaler', MinMaxScaler())
])
processor = ColumnTransformer([
    ('cat', cat_proc, cat_cols),
    ('num', nnum_proc, num_cols)
])

In [36]:
encode_train = processor.fit_transform(df_train)
df_train_tf = pd.DataFrame(encode_train, columns=processor.get_feature_names_out())
encode_test = processor.transform(df_test)
df_test_tf = pd.DataFrame(encode_test, columns=processor.get_feature_names_out())

In [37]:
display(df_train_tf)
display(df_test_tf)

,cat__col_b_dog,num__col_a,num__col_c
0,0.0,0.0,0.000000
1,1.0,1.0,0.333333
2,1.0,0.5,0.333333
3,1.0,0.0,0.333333
4,1.0,1.0,1.000000


,cat__col_b_dog,num__col_a,num__col_c
0,1.0,0.5,0.333333
1,1.0,0.0,0.000000
2,0.0,0.5,0.333333
